Charger les variables :

In [3]:
%store -r  RG_NAME SQL_SERVER_NAME SQL_DB_NAME

Récuperer et stocker les informations de l'utilisateur connecté :

In [29]:
%%bash

{
    echo "AZURE_SUBSCRIPTION_ID=\"$(az account show --query id -o tsv)\""
    echo "AZURE_TENANT_ID=\"$(az account show --query tenantId -o tsv)\""
    echo "USER_PRINCIPAL_NAME=\"$(az ad signed-in-user show --query userPrincipalName -o tsv)\""
    echo "USER_ID=\"$(az ad signed-in-user show --query id -o tsv)\""
    echo ""
} >> .env


Créer le serveur SQL :

In [19]:
%%bash -s "$RG_NAME" "$SQL_SERVER_NAME" 
source .env

az sql server create \
    --name "$2" \
    --resource-group "$1" \
    --location "FranceCentral" \
    --enable-ad-only-auth \
    --external-admin-principal-type User \
    --external-admin-name "$USER_PRINCIPAL_NAME" \
    --external-admin-sid "$USER_ID" \
    --query "state" \
    -o tsv

Ready


Créer la base de données :

In [ ]:
%%bash -s $RG_NAME $SQL_SERVER_NAME $SQL_DB_NAME
source .env

az login --use-device-code

az sql db create \
    --resource-group $1 \
    --server $2 \
    --name $3 \
    --edition GeneralPurpose \
    --family Gen5 \
    --capacity 1 \
    --compute-model Serverless \
    --auto-pause-delay 60 \
    --max-size 2GB \
    --subscription "$AZURE_SUBSCRIPTION_ID" \
    --user "$USER_PRINCIPAL_NAME" \
    --yes  \
    --query "status" \
    -o tsv

    Note : Les paramètres passés correspondent à une bdd avec un workload-environment "Development" sur le portal    